# Análise de Vendas - App de Delivery "Sabor Express"

In [1]:
import pandas as pd
from google.colab import files
import io

# Pede para você fazer upload dos arquivos
print("Por favor, faça o upload dos DOIS arquivos: 'pedidos_outubro.csv' e 'cardapio_restaurantes.csv'")
uploaded = files.upload()

# Lê os arquivos que você subiu
pedidos_df = pd.read_csv(io.BytesIO(uploaded['pedidos_outubro.csv']))
cardapio_df = pd.read_csv(io.BytesIO(uploaded['cardapio_restaurantes.csv']))

print("\nArquivos carregados com sucesso!")

# Corrigindo a data
pedidos_df['order_date'] = pd.to_datetime(pedidos_df['order_date'])

print("\nDataFrame de Pedidos:")
display(pedidos_df.head())
print("\nDataFrame do Cardápio:")
display(cardapio_df.head())

Por favor, faça o upload dos DOIS arquivos: 'pedidos_outubro.csv' e 'cardapio_restaurantes.csv'


Saving cardapio_restaurantes.csv to cardapio_restaurantes.csv
Saving pedidos_outubro.csv to pedidos_outubro.csv

Arquivos carregados com sucesso!

DataFrame de Pedidos:


,order_id,customer_id,item_id,order_date
0,10001,C101,101,2025-10-01
1,10002,C102,201,2025-10-01
2,10003,C101,102,2025-10-02
3,10004,C103,301,2025-10-02
4,10005,C104,201,2025-10-03



DataFrame do Cardápio:


,item_id,item_name,restaurant_name,price_eur
0,101,Pizza Margherita,Cantina Italiana,14.5
1,102,Pizza Pepperoni,Cantina Italiana,16.0
2,201,Hambúrguer Clássico,Burger Joint,12.0
3,202,Batatas Fritas,Burger Joint,4.5
4,301,Pad Thai,Sabores da Ásia,15.0


In [2]:
# Este é o comando mais importante deste projeto!
# Vamos "mesclar" (merge) os dois DataFrames.
# Usamos 'item_id' como a "chave" para conectar os dois.

df_completo = pd.merge(pedidos_df, cardapio_df, on='item_id', how='left')

print("DataFrames unidos com sucesso! Veja o resultado:")
display(df_completo.head())

DataFrames unidos com sucesso! Veja o resultado:


,order_id,customer_id,item_id,order_date,item_name,restaurant_name,price_eur
0,10001,C101,101,2025-10-01,Pizza Margherita,Cantina Italiana,14.5
1,10002,C102,201,2025-10-01,Hambúrguer Clássico,Burger Joint,12.0
2,10003,C101,102,2025-10-02,Pizza Pepperoni,Cantina Italiana,16.0
3,10004,C103,301,2025-10-02,Pad Thai,Sabores da Ásia,15.0
4,10005,C104,201,2025-10-03,Hambúrguer Clássico,Burger Joint,12.0


# 3. Análise Financeira (com o DataFrame completo)

In [3]:
# Pergunta 1: Faturamento total
faturamento_total = df_completo['price_eur'].sum()
print(f"Faturamento total no período: EUR {faturamento_total:,.2f}")

# Pergunta 2: Faturamento por restaurante
print("\nFaturamento total por restaurante:")
faturamento_restaurante = df_completo.groupby('restaurant_name')['price_eur'].sum().sort_values(ascending=False)
print(faturamento_restaurante.map('EUR {:,.2f}'.format))

# Pergunta 3: Ticket médio (valor médio por pedido)
ticket_medio = df_completo['price_eur'].mean()
print(f"\nTicket médio por pedido: EUR {ticket_medio:,.2f}")

# Pergunta 4: Item que gerou mais receita
print("\nReceita por item:")
receita_item = df_completo.groupby('item_name')['price_eur'].sum().sort_values(ascending=False)
print(receita_item.map('EUR {:,.2f}'.format))

Faturamento total no período: EUR 207.50

Faturamento total por restaurante:
restaurant_name
Cantina Italiana      EUR 75.50
Burger Joint          EUR 73.50
Sabores da Ásia       EUR 50.00
Padaria Pão Quente     EUR 8.50
Name: price_eur, dtype: object

Ticket médio por pedido: EUR 10.38

Receita por item:
item_name
Hambúrguer Clássico    EUR 60.00
Pad Thai               EUR 45.00
Pizza Margherita       EUR 43.50
Pizza Pepperoni        EUR 32.00
Batatas Fritas         EUR 13.50
Coxinha                 EUR 5.00
Rolinho Primavera       EUR 5.00
Suco de Laranja         EUR 3.50
Name: price_eur, dtype: object


# 4. Análise de Clientes e Popularidade

In [4]:
# Pergunta 5: Item mais vendido (em quantidade)
print("\nItem mais vendido (em quantidade):")
item_mais_vendido = df_completo['item_name'].value_counts()
print(item_mais_vendido)

# Pergunta 6: Clientes que gastaram mais (Top 3)
print("\nTop 3 Clientes por Gasto:")
top_clientes = df_completo.groupby('customer_id')['price_eur'].sum().sort_values(ascending=False).head(3)
print(top_clientes.map('EUR {:,.2f}'.format))

# Pergunta 7: Pedidos por restaurante
print("\nTotal de pedidos por restaurante:")
pedidos_restaurante = df_completo['restaurant_name'].value_counts()
print(pedidos_restaurante)


Item mais vendido (em quantidade):
item_name
Hambúrguer Clássico    5
Pizza Margherita       3
Pad Thai               3
Batatas Fritas         3
Pizza Pepperoni        2
Coxinha                2
Rolinho Primavera      1
Suco de Laranja        1
Name: count, dtype: int64

Top 3 Clientes por Gasto:
customer_id
C101    EUR 72.00
C102    EUR 37.00
C103    EUR 34.50
Name: price_eur, dtype: object

Total de pedidos por restaurante:
restaurant_name
Burger Joint          8
Cantina Italiana      5
Sabores da Ásia       4
Padaria Pão Quente    3
Name: count, dtype: int64


# 5. Desafios

In [5]:
# Desafio 1: Clientes da "Cantina Italiana"
print("--- Desafio 1: Clientes da Cantina Italiana ---")
clientes_italiana = df_completo.query('restaurant_name == "Cantina Italiana"')
display(clientes_italiana)

# Desafio 2: Criar coluna "dia_semana"
print("\n--- Desafio 2: Adicionando dia da semana ---")
df_completo['dia_semana'] = df_completo['order_date'].dt.day_name()
display(df_completo.head())

# Desafio 3: Salvar faturamento por restaurante
faturamento_restaurante_df = df_completo.groupby('restaurant_name')['price_eur'].sum().sort_values(ascending=False).reset_index()
faturamento_restaurante_df.to_csv('faturamento_por_restaurante.csv', index=False)
print("\nArquivo 'faturamento_por_restaurante.csv' salvo no ambiente do Colab!")

--- Desafio 1: Clientes da Cantina Italiana ---


,order_id,customer_id,item_id,order_date,item_name,restaurant_name,price_eur
0,10001,C101,101,2025-10-01,Pizza Margherita,Cantina Italiana,14.5
2,10003,C101,102,2025-10-02,Pizza Pepperoni,Cantina Italiana,16.0
7,10008,C101,101,2025-10-04,Pizza Margherita,Cantina Italiana,14.5
10,10011,C102,102,2025-10-06,Pizza Pepperoni,Cantina Italiana,16.0
16,10017,C103,101,2025-10-10,Pizza Margherita,Cantina Italiana,14.5



--- Desafio 2: Adicionando dia da semana ---


,order_id,customer_id,item_id,order_date,item_name,restaurant_name,price_eur,dia_semana
0,10001,C101,101,2025-10-01,Pizza Margherita,Cantina Italiana,14.5,Wednesday
1,10002,C102,201,2025-10-01,Hambúrguer Clássico,Burger Joint,12.0,Wednesday
2,10003,C101,102,2025-10-02,Pizza Pepperoni,Cantina Italiana,16.0,Thursday
3,10004,C103,301,2025-10-02,Pad Thai,Sabores da Ásia,15.0,Thursday
4,10005,C104,201,2025-10-03,Hambúrguer Clássico,Burger Joint,12.0,Friday



Arquivo 'faturamento_por_restaurante.csv' salvo no ambiente do Colab!
